In [ ]:
###### Applications Lab #1-- ATOC7500 Objective Analysis - bootstrapping
##### Originally coded by Prof. Kay (CU) with input from Vineel Yettella (CU ATOC Ph.D. 2018)
##### last updated August 25, 2020

###LEARNING GOALS:
###1) Working in an ipython notebook: read in csv file, make histogram plot
###2) Assessing statistical significance using bootstrapping (and t-test)

### GENERAL SETUP
%matplotlib inline  
# this enables plotting within notebook

import matplotlib   # library for plotting
import matplotlib.pyplot as plt #  later you will type plt.$COMMAND
import numpy as np   # basic math library  you will type np.$STUFF  e.g., np.cos(1)
import pandas as pd  # library for data analysis for text files (everything but netcdf files)
import scipy.stats as stats # imports stats functions https://docs.scipy.org/doc/scipy/reference/stats.html 

In [ ]:
### Read in the data
filename='snow_enso_data.csv'
data=pd.read_csv(filename,sep=',')
data.head()

In [ ]:
### Print the data column names
print(data.columns[0])
print(data.columns[1])
print(data.columns[2])

In [ ]:
### Print the data values - LOOK AT YOUR DATA.  If new to Python - check out what happens when you remove .values.
print(data['Year'].values)
print(data['LovelandPass_April1SWE_inches'].values)
print(data['Nino34_anomaly_prevDec'].values)

In [ ]:
### Calculate the average snowfall on April 1 at Loveland Pass, Colorado
SWE_avg=data['LovelandPass_April1SWE_inches'].mean()
SWE_std=data['LovelandPass_April1SWE_inches'].std()
N_SWE=len(data.LovelandPass_April1SWE_inches)
print('Average SWE (inches):',np.str(np.round(SWE_avg,2)))
print('Standard Deviation SWE (inches):',np.str(np.round(SWE_std,2)))
print('N:',np.str(N_SWE))

In [ ]:
### Print to figure out how to condition and make sure it is working.  Check out if new to Python.
#print(data.Nino34_anomaly_prevDec>1) ## this gives True/False
#print(data[data.Nino34_anomaly_prevDec>1])  ## where it is True, values will print

### Calculate the average SWE when it was an el nino year
SWE_avg_nino=data[data.Nino34_anomaly_prevDec>1.0]['LovelandPass_April1SWE_inches'].mean()
SWE_std_nino=data[data.Nino34_anomaly_prevDec>1.0]['LovelandPass_April1SWE_inches'].std()
N_SWE_nino=len(data[data.Nino34_anomaly_prevDec>1.0].LovelandPass_April1SWE_inches)
print('Average SWE El Nino (inches):',np.str(np.round(SWE_avg_nino,2)))
print('Standard Deviation SWE El Nino (inches):',np.str(np.round(SWE_std_nino,2)))
print('N El Nino:',np.str(N_SWE_nino))

In [ ]:
### Calculate the average SWE when it was an la nina year
SWE_avg_nina=data[data.Nino34_anomaly_prevDec<-1.0]['LovelandPass_April1SWE_inches'].mean()
SWE_std_nina=data[data.Nino34_anomaly_prevDec<-1.0]['LovelandPass_April1SWE_inches'].std()
N_SWE_nina=len(data[data.Nino34_anomaly_prevDec<-1.0].LovelandPass_April1SWE_inches)
print('Average SWE La Nina (inches):',np.str(np.round(SWE_avg_nina,2)))
print('Standard Deviation SWE La Nina (inches):',np.str(np.round(SWE_std_nina,2)))
print('N El Nino:',np.str(N_SWE_nina))

In [ ]:
### Bootstrap!!  Generate random samples of size N_SWE_nino and N_SWE_nina.  Do it once to see if it works.
P_random=np.random.choice(data.LovelandPass_April1SWE_inches,N_SWE_nino)
print(P_random)  ## LOOK AT YOUR DATA

In [ ]:
### Now Bootstrap Nbs times to generate a distribution of randomly selected mean SWE.
Nbs=1000
## initialize array
P_Bootstrap=np.empty((Nbs,N_SWE_nino))
## loop over to fill in array with randomly selected values
for ii in range(Nbs):
    P_Bootstrap[ii,:]=np.random.choice(data.LovelandPass_April1SWE_inches,N_SWE_nino)

## Calculate the means of your randomly selected SWE values.
P_Bootstrap_mean=np.mean(P_Bootstrap,axis=1)
print(len(P_Bootstrap_mean))  ## check length to see if you averaged across the correct axis
print(np.shape(P_Bootstrap_mean)) ## another option to look at the dimensions of a variable

P_Bootstrap_mean_avg=np.mean(P_Bootstrap_mean)
print(P_Bootstrap_mean_avg)
P_Bootstrap_mean_std=np.std(P_Bootstrap_mean)
print(P_Bootstrap_mean_std)
P_Bootstrap_mean_min=np.min(P_Bootstrap_mean)
print(P_Bootstrap_mean_min)
P_Bootstrap_mean_max=np.max(P_Bootstrap_mean)
print(P_Bootstrap_mean_max)

In [ ]:
### Use matplotlib to plot a histogram of the bootstrapped means to compare to the conditioned SWE mean
binsize=0.1
min4hist=np.round(np.min(P_Bootstrap_mean),1)-binsize
max4hist=np.round(np.max(P_Bootstrap_mean),1)+binsize
nbins=int((max4hist-min4hist)/binsize)

plt.hist(P_Bootstrap_mean,nbins,edgecolor='black')
plt.xlabel('Mean SWE (inches)');
plt.ylabel('Count');
plt.title('Bootstrapped Randomly Selected Mean SWE Values');

In [ ]:
## What is the probability that the snowfall was lower during El Nino by chance?
## Using Barnes equation (83) on page 15 to calculate probability using z-statistic
sample_mean=SWE_avg_nino
sample_N=1
population_mean=np.mean(P_Bootstrap_mean)
population_std=np.std(P_Bootstrap_mean)
xstd=population_std/np.sqrt(sample_N)
z_nino=(sample_mean-population_mean)/xstd
print("sample_mean - El Nino: ",np.str(np.round(sample_mean,2)))
print("population_mean: ",np.str(np.round(population_mean,2)))
print("population_std: ",np.str(np.round(population_std,2)))
print("Z-statistic (number of standard errors that the sample mean deviates from the population mean:")
print(np.round(z_nino,2))
prob=(1-stats.norm.cdf(np.abs(z_nino)))*100 ##this is a one-sided test
print("Probability one-tailed test (percent):")
print(np.round(prob,2)) 

In [ ]:
## What is the probability that the snowfall that the El Nino mean differs from the mean by chance?
## Using Barnes equation (83) on page 15 to calculate probability using z-statistic
sample_mean=SWE_avg_nino
sample_N=1
population_mean=np.mean(P_Bootstrap_mean)
population_std=np.std(P_Bootstrap_mean)
xstd=population_std/np.sqrt(sample_N)
z_nino=(sample_mean-population_mean)/xstd
print("sample_mean - El Nino: ",np.str(np.round(sample_mean,2)))
print("population_mean: ",np.str(np.round(population_mean,2)))
print("population_std: ",np.str(np.round(population_std,2)))
print("Z-statistic (number of standard errors that the sample mean deviates from the population mean):")
print(np.round(z_nino,2))
prob=(1-stats.norm.cdf(np.abs(z_nino)))*2*100 ##this is a two-sided test
print("Probability - two-tailed test (percent):")
print(np.round(prob,2)) 

In [ ]:
## What is the probability that the snowfall was higher during La Nina just due to chance?
## Using Barnes equation (83) on page 15 to calculate probability using z-statistic
sample_mean=SWE_avg_nina
sample_N=1
population_mean=np.mean(P_Bootstrap_mean)
population_std=np.std(P_Bootstrap_mean)
xstd=population_std/np.sqrt(sample_N)
z_nina=(sample_mean-population_mean)/xstd

print("sample_mean - La Nina: ",np.str(np.round(sample_mean,2)))
print("population_mean: ",np.str(np.round(population_mean,2)))
print("population_std: ",np.str(np.round(population_std,2)))
print("Z-statistic (number of standard errors that the sample mean deviates from the population mean:")
print(np.round(z_nina,2))
prob=(1-stats.norm.cdf(np.abs(z_nina)))*100 ##this is a one-sided test
print("Probability one-tailed test (percent):")
print(np.round(prob,2)) 

In [ ]:
## What is the probability that the snowfall during La Nina differed just due to chance?
## Using Barnes equation (83) on page 15 to calculate probability using z-statistic
sample_mean=SWE_avg_nina
sample_N=1
population_mean=np.mean(P_Bootstrap_mean)
population_std=np.std(P_Bootstrap_mean)
xstd=population_std/np.sqrt(sample_N)
z_nina=(sample_mean-population_mean)/xstd

print("sample_mean - La Nina: ",np.str(np.round(sample_mean,2)))
print("population_mean: ",np.str(np.round(population_mean,2)))
print("population_std: ",np.str(np.round(population_std,2)))
print("Z-statistic (number of standard errors that the sample mean deviates from the population mean):")
print(np.round(z_nina,2))
prob=(1-stats.norm.cdf(np.abs(z_nina)))*2*100 ##this is a two-sided test
print("Probability - two-tailed test (percent):")
print(np.round(prob,2)) 

In [ ]:
### Strategy #2:  Forget bootstrapping, let's use a t-test...
## Apply a t-test to test the null hypothesis that the means of the two samples 
## are the same at the 95% confidence level (alpha=0.025, two-sided test)
## If pvalue < alpha - reject null hypothesis.
print('Null Hypothesis:  ENSO snow years have the same mean as the full record.')
stats.ttest_ind(data['LovelandPass_April1SWE_inches'],data[data.Nino34_anomaly_prevDec>1.0]['LovelandPass_April1SWE_inches'])
print(stats.ttest_ind(data['LovelandPass_April1SWE_inches'],data[data.Nino34_anomaly_prevDec>1.0]['LovelandPass_April1SWE_inches']))
print('Cannot reject the null hypthesis.')

In [ ]:
### Strategy #3 (provided by Vineel Yettella)
SWE = data['LovelandPass_April1SWE_inches']
SWE_nino = data[data.Nino34_anomaly_prevDec>1.0]['LovelandPass_April1SWE_inches']

#We start by setting up a null hypothesis H0. 
#Our H0 will be that the difference in means of the two populations that the samples came from is equal to zero.
#We will use the bootstrap to test this null hypothesis.

#We next choose a significance level for the hypothesis test
alpha = 0.05

#All hypothesis tests need a test statistic.
#Here, we'll use the difference in sample means as the test statistic.
#create array to hold bootstrapped test statistic values
bootstrap_statistic = np.empty(10000)

#bootstrap 10000 times
for i in range(1,10000):
    
    #create a resample of SWE by sampling with replacement (same length as SWE)
    resample_original = np.random.choice(SWE, len(SWE), replace=True)
    
    #create a resample of SWE_nino by sampling with replacement (same length as SWE_nino)
    resample_nino = np.random.choice(SWE_nino, len(SWE_nino), replace=True)
    
    #Compute the test statistic from the resampled data, i.e., the difference in means
    bootstrap_statistic[i] = np.mean(resample_original) - np.mean(resample_nino)

#Let's plot the distribution of the test statistic
plt.hist(bootstrap_statistic,[-5,-4,-3,-2,-1,0,1,2,3,4,5],edgecolor='black')
plt.xlabel('Difference in sample means')
plt.ylabel('Count')
plt.title('Bootstrap distribution of difference in sample means')

#Create 95% CI from the bootstrapped distribution. The upper limit of the CI is defined as the 97.5% percentile
#and the lower limit as the 2.5% percentile of the boostrap distribution, so that 95% of the 
#distribution lies within the two limits

CI_up = np.percentile(bootstrap_statistic, 100*(1 - alpha/2.0))
CI_lo = np.percentile(bootstrap_statistic, 100*(alpha/2.0))

print(CI_up)
print(CI_lo)

#We see that the confidence interval contains zero, so we fail to reject the null hypothesis that the difference
#in means is equal to zero